In [1]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
import numpy as np 
from progressbar import progressbar

# load an image from filepath
def load_image(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return np.asarray(img)

# extract features from each photo in the directory
def extract_features(directory):
  # load the model
  model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
  print(model.summary())
  # extract features from each photo
  features = dict()

  for name in progressbar(listdir(directory)):

    filename = directory + '/' + name
    image = load_image(filename)
    # extract features
    feature = model.predict(image, verbose=0)
    # get image id
    image_id = name.split('.')[0]
    # store feature
    features[image_id] = feature
    print('>%s' % name)
    
  return features
# extract features from all images

directory = 'Flickr8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

Using TensorFlow backend.
N/A% (0 of 8091) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

  0% (1 of 8091) |                       | Elapsed Time: 0:00:00 ETA:   1:55:32

>2387197355_237f6f41ee.jpg


  0% (2 of 8091) |                       | Elapsed Time: 0:00:01 ETA:   2:06:20

...

  0% (6 of 8091) |                       | Elapsed Time: 0:00:05 ETA:   1:54:12

>3421131122_2e4bde661e.jpg


  0% (7 of 8091) |                       | Elapsed Time: 0:00:06 ETA:   1:57:21

>3229730008_63f8ca2de2.jpg


KeyboardInterrupt: 